In [1]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import gym_kilobots

### Create environment and train

In [2]:
# Define four objects in a centered square
square_objects = [
    ((-0.25,  0.25), 0.0),   # top-left
    (( 0.25,  0.25), 0.0),   # top-right
    (( 0.25, -0.25), 0.0),   # bottom-right
    ((-0.25, -0.25), 0.0),   # bottom-left
]

# Define 1 object in the center of the environment
single_center_object = [
    ((0.0, 0.0), 0.0)        # center,
]

# Choose a fixed light position
light_pos = (-0.5, 0.5)

# Choose four explicit kilobot positions
kb_positions = [
    (-0.50, 0.60),
    (-0.50, 0.60),
    (-0.60, 0.50),
    (-0.60, 0.50),
]

# NOTE: if you don't do this, the environment will randomly place the objects
env = gym.make(
    'Kilobots-QuadAssembly-v0',
    render_mode='human',
    num_kilobots=len(kb_positions),
    object_config=single_center_object,
    light_position=light_pos,
    kilobot_positions=kb_positions
)


In [3]:
check_env(env,warn=True)

In [4]:
#vec_env = make_vec_env(env, n_envs = 1)

# Train the agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=500_000)
model.save("swarm_ppo")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 31.2     |
|    ep_rew_mean     | 25       |
| time/              |          |
|    fps             | 988      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 49.7        |
|    ep_rew_mean          | 29.8        |
| time/                   |             |
|    fps                  | 903         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008652613 |
|    clip_fraction        | 0.0593      |
|    clip_range           | 0.2         |
|    entropy_loss   

In [5]:
model = PPO.load("swarm_ppo")

def controller(x):
      action, _ = model.predict(x)
      return action

# Create a separate evaluation environment
eval_gym = gym.make(
    'Kilobots-QuadAssembly-v0',
    render_mode='human',
    num_kilobots=len(kb_positions),
    object_config=single_center_object,
    light_position=light_pos,
    kilobot_positions=kb_positions
)

running = True
obs, info = eval_gym.reset()

while running:
      eval_gym.render()

      x = obs
      action = controller(x)

      obs, reward, terminated, truncated, info = eval_gym.step(action)

      # Print the orientation of the objects using the unwrapped environment
      print(eval_gym.unwrapped.get_objects_status())

      # End episode
      if terminated or truncated:
            break

eval_gym.close()  # Ensure the environment is properly closed

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
['object 0: position: [0. 0.], orientation: 0.0']
